[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/twMr7/Python-Machine-Learning/blob/master/09-Other_Utilities.ipynb)

# 9. 其他實用工具 Other Utilities

Python 的通用性來自於豐富的標準函式庫，本章介紹以下幾種常用的工具模組。
+ [**9.1 日期與時間（Date and Time）**](#module-datetime)
+ [**9.2 物件序列化（Python Object Serialization）**](#module-pickle)
+ [**9.3 亂數（Random Numbers）**](#module-random)
+ [**9.4 數學函數（Math Functions）**](#module-math)
+ [**9.5 檔案系統路徑（File System Paths）**](#module-pathlib)


<a id="module-datetime"></a>

## 9.1 日期與時間 Date and Time

Python 標準函式庫中的 [`datetime`](https://docs.python.org/3/library/datetime.html#datetime-objects) 模組可以用來處理日期時間相關的資料，包含了 `date`, `time`, `datetime`, `timedelta`, `timezone` 等型別。

In [ ]:
# 載入 datetime 模組
from datetime import datetime

In [ ]:
# 現在的日期時間，返回 datetime 型別
t1 = datetime.now()
# 轉成字串 str 型別
str(t1)

In [ ]:
# 從日期時間的字串轉成 datetime 型別
tformat = '%Y-%m-%d %H:%M:%S'
t2 = datetime.strptime('2018-10-28  15:10:00', tformat)

# 比較兩個 datetime
if (t1 > t2):
    print('t1 比 t2 晚', t1 - t2)
else:
    print('t2 比 t1 晚', t2 - t1)

<a id="module-pickle"></a>

## 9.2 物件序列化 Python Object Serialization

Python 標準函式庫中的 [`pickle`](https://docs.python.org/3/library/pickle.html#module-pickle) 模組，提供了將 Python 物件序列化（serializing）及解序列化（de-serializing）的方法。 序列化指的是將物件階層轉換成位元組串流（byte stream），以方便物件的儲存、網路傳送、以及不同平臺的互通交換，反向的解序列化操作則是將位元組串流轉換成物件階層。

+ `pickle` 模組可以將物件儲存至檔案，或從檔案載入物件，檔案的存取需使用 binary 模式。
+ `pickle` 模組提供的序列化功能只適用於 Python 物件專用，標準函式庫中另外有跨平臺及程式語言的通用型的序列化模組 [`json`](https://docs.python.org/3/library/json.html#module-json)，但 `json` 只支援較少的 Python 內建物件型別。

In [ ]:
# 載入 pickle 模組
import pickle

In [ ]:
# 建立一個數據記錄的結構
record = [
    {'時間':datetime.strptime('2019-04-03 10:35:58', tformat), '體溫':37.0, '速度':35.0, '心率':92},
    {'時間':datetime.strptime('2019-04-03 10:37:00', tformat), '體溫':37.1, '速度':33.8, '心率':97},
    {'時間':datetime.strptime('2019-04-03 10:37:59', tformat), '體溫':37.4, '速度':35.5, '心率':99}
]

# 開啟新的 binary 檔案，用 pickle 將 record 物件 serialize
pfile = open('record.pkl', 'wb')
pickle.dump(record, pfile)
pfile.close()

# 讀入檔案，將 record 物件 de-serialize
pfile = open('record.pkl', 'rb')
record2 = pickle.load(pfile)
pfile.close()
record2

<a id="module-random"></a>

## 9.3 亂數 Random Numbers

Python 標準函式庫中的 [`random`](https://docs.python.org/3/library/random.html) 模組，提供了擬隨機（pseudo-random）亂數產生的方法。

+ `random()` - 返回下一個 [0.0, 1.0) 區間內的隨機實數。
+ `randrange(start, stop[, step])` - 返回下一個 [start, stop) 區間內的隨機整數。
+ `randint(a, b)` - 返回下一個 [a, b] 區間內的隨機整數，同 `randrange(a, b+1)`。
+ `choice(seq)` - 從 seq 序列中隨機選取其中一個成員。
+ `shuffle(seq)` - 將 seq 序列中的元素順序重新隨機排列，序列必須是可就地變更的容器類別。
+ `sample(seq, k)` - 從 seq 序列或集合中，返回隨機選取 k 個樣本的 List 清單。

In [ ]:
# 載入 random 模組
import random

In [ ]:
# 產生 100 個隨機實數數列
Lr = [random.random() for x in range(100)]
# 產生 100 個隨機整數數列
Li = [random.randint(1, 100) for x in range(100)]
# 從數列中隨機選取 10 個樣本，產生新的隨機數列
[x * y for x, y in zip(random.sample(Lr, 10), random.sample(Li, 10))]

<a id="module-math"></a>

## 9.4 數學函數 Math Functions

Python 標準函式庫中的 [`math`](https://docs.python.org/3/library/math.html) 模組，提供了用於實數運算的常用函數。

In [ ]:
# 載入 math 模組
import math

In [ ]:
# 內建函式的 sum() 在浮點數運算的精度不足
print(sum([.1, .1, .1, .1, .1, .1, .1, .1, .1, .1]))
# math 模組的 fsum() 可避免精度的誤差
print(math.fsum([.1, .1, .1, .1, .1, .1, .1, .1, .1, .1]))

In [ ]:
# cosine 180 度
print('cosine(pi) =', math.cos(math.pi))
# sine 90 度
print('sine(pi/2) =', math.sin(math.radians(90)))

<a id="module-pathlib"></a>

## 9.5 檔案系統路徑 File System Paths

Python 標準函式庫中的 [`pathlib`](https://docs.python.org/3/library/pathlib.html) 模組，提供了通用於不同平台的檔案系統路徑操作，`Path` 物件可以比較、解析路徑的組成部份、也可以串接重組，主要有以下屬性：

+ `Path.drive` - 目標路徑的磁碟代號
+ `Path.root` - 目標路徑的根目錄
+ `Path.parent` - 目標路徑的上層目錄
+ `Path.name` - 目標路徑最後部份的名字
+ `Path.suffix` - 目標路徑最後部份的副檔名
+ `Path.stem` - 目標路徑最後部份去除副檔名的名字

常用的 `Path` 類別方法如下：

+ `Path.cwd()` - 目前工作目錄。
+ `Path.home()` - 登入使用者的家目錄。
+ `Path(str)` - 從字串 str 建立路徑物件。
+ `Path.exists()` - 路徑的檔案或目錄是否存在。
+ `Path.glob(pattern)` - 返回生成函式，用來列出路徑下符合指定 pattern 的所有檔案或目錄。
+ `Path.is_dir()` - 檢查路徑的目標是否爲目錄。
+ `Path.is_file()` - 檢查路徑的目標是否爲檔案。
+ `Path.iterdir()` - 當目標路徑爲目錄時，用來迭代尋訪目錄下的所有檔案。
+ `Path.mkdir()` - 當目標路徑爲目錄時，爲該目標建立目錄。
+ `Path.rename(new_name)` - 重新命名檔案。
+ `Path.open(mode)` - 功能同內建函式 `open()`，使用指定模式開啓檔案，返回檔案物件。 


In [ ]:
# 載入 Path 類別
from pathlib import Path

In [ ]:
# 列出目前工作目錄下所有的檔案及目錄
pwd = Path.cwd()
print('Current working directory: ', pwd)
for f in pwd.iterdir():
    print(f.name)

# 建立一個記錄檔案名字與副檔名對照的字典
{f.stem:f.suffix for f in pwd.iterdir() if f.is_file()}

In [ ]:
# 列出目前工作目錄下所有副檔名是 .ipynb 的檔案
print([f.name for f in pwd.glob('*.ipynb')])
# 目前工作目錄下所有副檔名是 .ipynb 的路徑清單
list(pwd.glob('*.ipynb'))